In [ ]:
import random
from deap import base
from deap import creator
from deap import tools
from functools import partial
import numpy as np
import pickle
from deap import algorithms



In [7]:
def objective(individual, generation):
    return random.random()+individual[0],



In [67]:

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)


toolbox = base.Toolbox()
toolbox.register("attr_bool", random.uniform, 0.90, 1.10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("map", pool.map)
toolbox.register("mate", tools.cxTwoPoints)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", objective)
pop = toolbox.population(n=30)
CXPB, MUTPB = 0.5, 1.0

In [72]:
population = toolbox.population(n=30)
start_gen = 0
halloffame = tools.HallOfFame(maxsize=1, similar=np.array_equal)
logbook = tools.Logbook()



#stats = tools.Statistics(lambda ind: ind.fitness.values)
stats_fit = tools.Statistics(lambda ind: ind[1])
stats_ind0 = tools.Statistics(lambda ind: ind[0])
mstats = tools.MultiStatistics(fitness=stats_fit, individual0=stats_ind0)

# stats_size = tools.Statistics(key=len)

listt=[stats_fit, stats_ind0]
for stats in listt:
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("size", np.size)

# stats.register("avg", np.mean)
# stats.register("std", np.std)
# stats.register("min", np.min)
# stats.register("max", np.max)
# stats.register("size", np.size)



FREQ=1
for gen in range(start_gen, 100):
    population = algorithms.varAnd(population, toolbox, cxpb=CXPB, mutpb=MUTPB)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(partial(toolbox.evaluate, generation=gen), invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    halloffame.update(population)
    record = mstats.compile(population)
    logbook.record(gen=gen, evals=len(invalid_ind), **record)

    population = toolbox.select(population, k=len(population))
    print(logbook.stream)
    if gen % FREQ == 0:
        # Fill the dictionary using the dict(key=value[, ...]) constructor
        cp = dict(population=population, generation=gen, halloffame=halloffame,
                  logbook=logbook, rndstate=random.getstate())

        with open("checkpoint_name.pkl", "wb") as cp_file:
            pickle.dump(cp, cp_file)

    

     	   	                     fitness                     	                  individual0                  
     	   	-------------------------------------------------	-----------------------------------------------
evals	gen	avg    	max   	min     	size	std      	avg    	max    	min     	size	std    
30   	0  	1.03531	1.4278	0.905092	30  	0.0897334	1.05545	2.37448	0.923134	30  	0.25754
30   	1  	0.986849	2.00026	-0.83216	30  	0.430917 	1.28514	2.56486	0.923134	30  	0.566491
30   	2  	1.04713 	3.31882	0.189352	30  	0.508766 	1.81148	2.88572	0.940886	30  	0.728913
30   	3  	1.37276 	3.31882	0.931398	30  	0.785502 	2.31999	2.88572	0.992362	30  	0.361594
30   	4  	1.39281 	3.99408	1.01892 	30  	0.845295 	2.44545	2.88572	2.15589 	30  	0.128577
30   	5  	1.1282  	2.37127	0.525982	30  	0.352455 	2.51108	2.88572	2.37448 	30  	0.152201
30   	6  	1.20654 	2.37127	0.636566	30  	0.43503  	2.58289	2.88572	2.37448 	30  	0.181191
30   	7  	1.34157 	2.37127	0.949513	30  	0.514439 	2.61885	2.88572	2.3

30   	87 	1.56573 	1.56573	1.56573   	30  	6.66134e-16	28.4914	29.534 	27.4336 	30  	0.271366   
30   	88 	1.5482  	1.61174	0.993891  	30  	0.103263   	28.5288	29.2107	28.4899 	30  	0.140401   
30   	89 	1.55797 	2.6863 	0.212442  	30  	0.32069    	28.5996	29.3949	28.4899 	30  	0.225948   
30   	90 	1.56573 	1.56573	1.56573   	30  	6.66134e-16	28.9578	30.2371	28.4899 	30  	0.569824   
30   	91 	1.53947 	1.56573	0.777849  	30  	0.141429   	29.3268	30.2371	28.4899 	30  	0.711604   
30   	92 	1.50107 	1.61813	0.41345   	30  	0.252348   	29.9297	30.2371	28.4899 	30  	0.42503    
30   	93 	1.54305 	1.85457	0.725905  	30  	0.162281   	30.2001	30.2371	29.9274 	30  	0.0769083  
30   	94 	1.51688 	1.85457	0.588043  	30  	0.1887     	30.1681	30.2371	28.8132 	30  	0.257508   
30   	95 	1.64106 	2.37774	1.43629   	30  	0.216972   	30.2214	30.2371	30.1597 	30  	0.0179282  
30   	96 	1.59609 	2.37774	0.846115  	30  	0.25747    	30.2751	31.9685	29.4147 	30  	0.413028   
30   	97 	1.71642 	2.37774	1.5

In [62]:

print(logbook.select('gen'))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
